In [ ]:
# Install dependencies required for this example.
# If you're not using pip to install depdencies (for example, you're using conda or uv),
# skip this cell and and install using the package manager of your choice.
# Restart the notebook after installing dependencies.
%pip install "xarray[complete]>=2025.1.2" "zarr>=3.0.4" requests aiohttp
!apt-get install -y libproj-dev proj-data proj-bin libgeos-dev
!pip install cython
!pip install cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.9/203.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.3/147.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Don

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import pandas as pd
import numpy as np
import tempfile
import requests
import os

# Descargar shapefiles principales y de máscara
shapefiles = {
    "departamentos.shp": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.shp",
    "departamentos.dbf": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.dbf",
    "departamentos.shx": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.shx",
}
shapefiles_extra = {
    "SUDA_NUEVO_SIN_PERU_3.shp": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.shp",
    "SUDA_NUEVO_SIN_PERU_3.dbf": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.dbf",
    "SUDA_NUEVO_SIN_PERU_3.shx": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.shx",
}

tempdir = tempfile.mkdtemp()
for files in [shapefiles, shapefiles_extra]:
    for filename, url in files.items():
        response = requests.get(url)
        with open(os.path.join(tempdir, filename), "wb") as f:
            f.write(response.content)

gdf_peru = gpd.read_file(os.path.join(tempdir, "departamentos.shp"))
gdf_suda = gpd.read_file(os.path.join(tempdir, "SUDA_NUEVO_SIN_PERU_3.shp"))

# Dataset GEFS

ds = xr.open_zarr(
    "https://data.dynamical.org/noaa/gefs/forecast-35-day/latest.zarr?email=optional@email.com",
    decode_timedelta=True
)
latest_init = ds.init_time.max().values
print("Usando init_time:", pd.to_datetime(latest_init).strftime("%Y-%m-%d"))

ds_sub = ds.sel(init_time=latest_init).sel(latitude=slice(0.5, -19.5), longitude=slice(-82, -68))

# Extraer datos por miembro
ds_members = ds_sub.sel(lead_time=slice("0h", "27d"))
tmax_all = ds_members["maximum_temperature_2m"] - 273.15  # Convertir de K a °C
forecast_time = ds_members.init_time + ds_members.lead_time
tmax_all = tmax_all.assign_coords(forecast_time=forecast_time)
tmax_all = tmax_all.swap_dims({"lead_time": "forecast_time"})
tmax_daily_all = tmax_all.groupby("forecast_time.date").mean(dim="forecast_time")

semanas = {
    "SEMANA 1": slice(0, 7),
    "SEMANA 2": slice(7, 14),
    "SEMANA 3": slice(14, 21),
    "SEMANA 4": slice(21, 28),
}

# Paleta de colores personalizada para temperatura
colors = ['#00079F', '#0116C8', '#0125F0', '#013DFF', '#015AFE', '#0177FE', '#2699FE', '#58BCFE',
          '#8AE0FE', '#A8EAFE', '#C3F0FF', '#DEF7FF',  '#FFF6C1', '#FFF19D', '#FFDE5D', '#FFC50E',
          '#FF9501', '#FF6101', '#F62A00', '#E72200', '#D71A00', '#C81101', '#B80901', '#A90100']
cmap_custom = mcolors.ListedColormap(colors)
norm_custom = mcolors.BoundaryNorm(boundaries=np.arange(7, 32, 1), ncolors=len(colors))

# Figura por miembro
members = tmax_all.ensemble_member.values
for member in members:
    fig, axes = plt.subplots(1, 4, figsize=(20, 6), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
    fig.subplots_adjust(wspace=0.01, hspace=0)

    extent = [-82, -68, -19.5, 0.5]
    x_ticks = range(-82, -67, 3)
    y_ticks = range(-20, 1, 5)

    for i, (ax, (semana, rango)) in enumerate(zip(axes, semanas.items())):
        tmax_member = tmax_daily_all.sel(ensemble_member=member)
        tmax_avg = tmax_member.isel(date=rango).mean(dim="date")
        week_dates = tmax_member.date.values[rango]
        start_date = pd.to_datetime(str(week_dates[0])).strftime("%d-%b").upper()
        end_date = pd.to_datetime(str(week_dates[-1])).strftime("%d-%b").upper()

        ax.set_extent(extent, crs=ccrs.PlateCarree())
        ax.set_xticks(x_ticks, crs=ccrs.PlateCarree())
        ax.set_yticks(y_ticks, crs=ccrs.PlateCarree())
        ax.tick_params(labelbottom=True, labelleft=(i == 0))

        ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.add_feature(cfeature.LAND, facecolor='lightgray')

        mesh = ax.pcolormesh(tmax_avg.longitude, tmax_avg.latitude, tmax_avg, cmap=cmap_custom, norm=norm_custom, shading="nearest")
        gdf_peru.boundary.plot(ax=ax, edgecolor='black', linewidth=0.5, zorder=5)
        gdf_suda.plot(ax=ax, color='white', linewidth=0, zorder=10)

        ax.set_title(f"**{semana}**\n{start_date} – {end_date}", fontsize=11)

    cbar = fig.colorbar(mesh, ax=axes.ravel().tolist(), orientation="vertical", shrink=0.7, pad=0.03)
    cbar.set_label("Temperatura Máxima [°C]")

    plt.suptitle(f"TMAX SEMANAL - MIEMBRO {member} [GEFS]", fontsize=16, y=1.07)
    output_file = f"tmax_miembro_{member}_semanal_peru_{pd.to_datetime(latest_init).strftime('%Y%m%d')}.png"
    #plt.savefig(output_file, bbox_inches='tight', pad_inches=0.05)
    plt.close()
    print(f"Imagen guardada como: {output_file}")

# Promedio general
fig, axes = plt.subplots(1, 4, figsize=(20, 6), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
fig.subplots_adjust(wspace=0.01, hspace=0)
tmax_prom = tmax_daily_all.mean(dim="ensemble_member")

for i, (ax, (semana, rango)) in enumerate(zip(axes, semanas.items())):
    tmax_avg = tmax_prom.isel(date=rango).mean(dim="date")
    week_dates = tmax_prom.date.values[rango]
    start_date = pd.to_datetime(str(week_dates[0])).strftime("%d-%b").upper()
    end_date = pd.to_datetime(str(week_dates[-1])).strftime("%d-%b").upper()

    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.set_xticks(x_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(y_ticks, crs=ccrs.PlateCarree())
    ax.tick_params(labelbottom=True, labelleft=(i == 0))

    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')

    mesh = ax.pcolormesh(tmax_avg.longitude, tmax_avg.latitude, tmax_avg, cmap=cmap_custom, norm=norm_custom, shading="nearest")
    gdf_peru.boundary.plot(ax=ax, edgecolor='black', linewidth=0.5, zorder=5)
    gdf_suda.plot(ax=ax, color='white', linewidth=0, zorder=10)

    ax.set_title(f"**{semana}**\n{start_date} – {end_date}", fontsize=11)

cbar = fig.colorbar(mesh, ax=axes.ravel().tolist(), orientation="vertical", shrink=0.7, pad=0.03)
cbar.set_label("Temperatura Máxima [°C]")

plt.suptitle("TMAX SEMANAL PROMEDIO DE ENSEMBLES PARA PERÚ [GEFS]", fontsize=16, y=1.07)
output_file = f"tmax_ensmean_semanal_peru_{pd.to_datetime(latest_init).strftime('%Y%m%d')}.png"
plt.savefig(output_file, bbox_inches='tight', pad_inches=0.05)
plt.show()
print(f"Imagen guardada como: {output_file}")

Usando init_time: 2025-06-10


/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


Imagen guardada como: tmax_miembro_0_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_1_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_2_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_3_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_4_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_5_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_6_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_7_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_8_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_9_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_10_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_11_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_12_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_13_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_14_semanal_peru_20250610.png
Imagen guardada como: tmax_miembro_15_semanal_peru

In [ ]:
#Precipuitacion categorica

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import pandas as pd
import tempfile
import requests
import os

# 0️⃣ Descargar shapefiles
shapefiles = {
    "departamentos.shp": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.shp",
    "departamentos.dbf": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.dbf",
    "departamentos.shx": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.shx",
}

shapefiles_extra = {
    "SUDA_NUEVO_SIN_PERU_3.shp": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.shp",
    "SUDA_NUEVO_SIN_PERU_3.dbf": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.dbf",
    "SUDA_NUEVO_SIN_PERU_3.shx": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.shx",
}

tempdir = tempfile.mkdtemp()
for files in [shapefiles, shapefiles_extra]:
    for filename, url in files.items():
        response = requests.get(url)
        with open(os.path.join(tempdir, filename), "wb") as f:
            f.write(response.content)

# Leer shapefiles
gdf_peru = gpd.read_file(os.path.join(tempdir, "departamentos.shp"))
gdf_suda = gpd.read_file(os.path.join(tempdir, "SUDA_NUEVO_SIN_PERU_3.shp"))

# 1️⃣ Abrir dataset GEFS desde Zarr
ds = xr.open_zarr(
    "https://data.dynamical.org/noaa/gefs/forecast-35-day/latest.zarr?email=optional@email.com",
    decode_timedelta=True
)

# 2️⃣ Fecha más reciente
latest_init = ds.init_time.max().values
print("Usando init_time:", pd.to_datetime(latest_init).strftime("%Y-%m-%d"))

# 3️⃣ Subconjunto PERÚ y promedio ensemble
ds_mean = (
    ds.sel(init_time=latest_init)
    .sel(latitude=slice(0.5, -19.5), longitude=slice(-82, -68))
    .mean(dim="ensemble_member")
)

# 4️⃣ Semana 1 a 4: lead_time hasta 27 días
ds_weeks = ds_mean.sel(lead_time=slice("0h", "27d"))

# 5️⃣ Extraer variable categórica de lluvia
rain_cat = ds_weeks["categorical_rain_surface"]

# 6️⃣ Coordenada temporal real
forecast_time = ds_weeks.init_time + ds_weeks.lead_time
rain_cat = rain_cat.assign_coords(forecast_time=forecast_time)
rain_cat = rain_cat.swap_dims({"lead_time": "forecast_time"})

# 7️⃣ Agrupar por día calendario y promediar (frecuencia diaria)
rain_daily = rain_cat.groupby("forecast_time.date").mean(dim="forecast_time")

# 8️⃣ Definir semanas (en días)
semanas = {
    "SEMANA 1": slice(0, 7),
    "SEMANA 2": slice(7, 14),
    "SEMANA 3": slice(14, 21),
    "SEMANA 4": slice(21, 28),
}

# 9️⃣ Paleta de colores personalizada invertida
raw_colors = [ ... ]  # ← Aquí colocas tu lista completa de colores
colors_norm = [(r/255, g/255, b/255) for r, g, b in raw_colors]
cmap_custom = mcolors.ListedColormap(colors_norm[::-1])
norm_custom = mcolors.Normalize(vmin=0, vmax=100)

# 🔟 Crear figura con 4 subplots en una fila
fig, axes = plt.subplots(1, 4, figsize=(17, 6), subplot_kw={'projection': ccrs.PlateCarree()}, constrained_layout=True)
plt.subplots_adjust(wspace=0.02)

for i, (ax, (semana, rango)) in enumerate(zip(axes, semanas.items())):
    prec = rain_daily.isel(date=rango).mean(dim="date") * 100
    week_dates = rain_daily.date.values[rango]
    start_date = pd.to_datetime(str(week_dates[0])).strftime("%d-%b").upper()
    end_date = pd.to_datetime(str(week_dates[-1])).strftime("%d-%b").upper()

    ax.set_extent([-82, -68, -19.5, 0.5], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')

    gdf_peru.boundary.plot(ax=ax, edgecolor='black', linewidth=0.5)
    gdf_suda.boundary.plot(ax=ax, edgecolor='gray', linestyle='--', linewidth=0.4)

    if i == 0:
        ax.set_yticks(range(-20, 1, 5), crs=ccrs.PlateCarree())
        ax.set_xticks(range(-82, -67, 3), crs=ccrs.PlateCarree())
        ax.tick_params(labelleft=True, labelbottom=True)
    else:
        ax.tick_params(labelleft=False, labelbottom=True)
        ax.set_xticks(range(-82, -67, 3), crs=ccrs.PlateCarree())

    mesh = ax.pcolormesh(prec.longitude, prec.latitude, prec, cmap=cmap_custom, norm=norm_custom, shading="auto")
    ax.set_title(f"**{semana}**\n{start_date} – {end_date}", fontsize=11)

# Colorbar global
cbar = fig.colorbar(mesh, ax=axes.ravel().tolist(), orientation="vertical", shrink=0.7, pad=0.03)
cbar.set_label("Probabilidad de Lluvia [%]")

# Título general
plt.suptitle("FRECUENCIA SEMANAL DE LLUVIA (CATEGORICAL) PARA PERÚ [GEFS ENSEMBLE]", fontsize=16, y=1.07)

# Guardar imagen
fecha_str = pd.to_datetime(latest_init).strftime("%Y%m%d")
output_file = f"lluvia_categ_semanal_peru_{fecha_str}.png"
plt.savefig(output_file, dpi=300, bbox_inches='tight')
plt.show()

print(f"Imagen guardada como: {output_file}")


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import pandas as pd
import numpy as np
import tempfile
import requests
import os

# 1. Descargar shapefiles
shapefiles = {
    "departamentos.shp": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.shp",
    "departamentos.dbf": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.dbf",
    "departamentos.shx": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/DEPARTAMENTOS.shx",
}
shapefiles_extra = {
    "SUDA_NUEVO_SIN_PERU_3.shp": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.shp",
    "SUDA_NUEVO_SIN_PERU_3.dbf": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.dbf",
    "SUDA_NUEVO_SIN_PERU_3.shx": "https://raw.githubusercontent.com/Jorgellamocca/GitC3Swrf/main/shape/SUDA_NUEVO_SIN_PERU_3.shx",
}

tempdir = tempfile.mkdtemp()
for files in [shapefiles, shapefiles_extra]:
    for filename, url in files.items():
        response = requests.get(url)
        with open(os.path.join(tempdir, filename), "wb") as f:
            f.write(response.content)

gdf_peru = gpd.read_file(os.path.join(tempdir, "departamentos.shp"))
gdf_suda = gpd.read_file(os.path.join(tempdir, "SUDA_NUEVO_SIN_PERU_3.shp"))

# 2. Cargar dataset GEFS
ds = xr.open_zarr(
    "https://data.dynamical.org/noaa/gefs/forecast-35-day/latest.zarr?email=optional@email.com",
    decode_timedelta=True
)
latest_init = ds.init_time.max().values
print("Usando init_time:", pd.to_datetime(latest_init).strftime("%Y-%m-%d"))

# 3. Subset geográfico y lead time
ds_sub = ds.sel(init_time=latest_init).sel(latitude=slice(0.5, -19.5), longitude=slice(-82, -68))
ds_members = ds_sub.sel(lead_time=slice("0h", "27d"))

# 4. Preparar variable de nubosidad (%)
cloud_all = ds_members["total_cloud_cover_atmosphere"] * 100  # 0–100 %
forecast_time = ds_members.init_time + ds_members.lead_time
cloud_all = cloud_all.assign_coords(forecast_time=forecast_time)
cloud_all = cloud_all.swap_dims({"lead_time": "forecast_time"})

# 5. Promedio diario
cloud_daily_all = cloud_all.groupby("forecast_time.date").mean(dim="forecast_time")

# 6. Definir semanas
semanas = {
    "SEMANA 1": slice(0, 7),
    "SEMANA 2": slice(7, 14),
    "SEMANA 3": slice(14, 21),
    "SEMANA 4": slice(21, 28),
}

# 7. Configuración gráfica
extent = [-82, -68, -19.5, 0.5]
x_ticks = range(-82, -67, 3)
y_ticks = range(-20, 1, 5)

# 8. Graficar por miembro del ensamble
members = cloud_all.ensemble_member.values
for member in members:
    fig, axes = plt.subplots(1, 4, figsize=(20, 6), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
    fig.subplots_adjust(wspace=0.01, hspace=0)

    for i, (ax, (semana, rango)) in enumerate(zip(axes, semanas.items())):
        cloud_member = cloud_daily_all.sel(ensemble_member=member)
        cloud_avg = cloud_member.isel(date=rango).mean(dim="date")
        week_dates = cloud_member.date.values[rango]
        start_date = pd.to_datetime(str(week_dates[0])).strftime("%d-%b").upper()
        end_date = pd.to_datetime(str(week_dates[-1])).strftime("%d-%b").upper()

        ax.set_extent(extent, crs=ccrs.PlateCarree())
        ax.set_xticks(x_ticks, crs=ccrs.PlateCarree())
        ax.set_yticks(y_ticks, crs=ccrs.PlateCarree())
        ax.tick_params(labelbottom=True, labelleft=(i == 0))

        ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.add_feature(cfeature.LAND, facecolor='lightgray')

        mesh = ax.pcolormesh(cloud_avg.longitude, cloud_avg.latitude, cloud_avg,
                             cmap="Blues", shading="nearest", vmin=0, vmax=100)

        gdf_peru.boundary.plot(ax=ax, edgecolor='black', linewidth=0.5, zorder=5)
        gdf_suda.plot(ax=ax, color='white', linewidth=0, zorder=10)

        ax.set_title(f"**{semana}**\n{start_date} – {end_date}", fontsize=11)

    cbar = fig.colorbar(mesh, ax=axes.ravel().tolist(), orientation="vertical", shrink=0.7, pad=0.03)
    cbar.set_label("Cobertura Nubosa Total [%]")

    plt.suptitle(f"NUBOSIDAD SEMANAL - MIEMBRO {member} [GEFS]", fontsize=16, y=1.07)
    output_file = f"nubes_miembro_{member}_semanal_peru_{pd.to_datetime(latest_init).strftime('%Y%m%d')}.png"
    #plt.savefig(output_file, bbox_inches='tight', pad_inches=0.05)
    plt.close()
    print(f"Imagen guardada como: {output_file}")

# 9. Promedio general del ensamble
fig, axes = plt.subplots(1, 4, figsize=(20, 6), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
fig.subplots_adjust(wspace=0.01, hspace=0)
cloud_prom = cloud_daily_all.mean(dim="ensemble_member")

for i, (ax, (semana, rango)) in enumerate(zip(axes, semanas.items())):
    cloud_avg = cloud_prom.isel(date=rango).mean(dim="date")
    week_dates = cloud_prom.date.values[rango]
    start_date = pd.to_datetime(str(week_dates[0])).strftime("%d-%b").upper()
    end_date = pd.to_datetime(str(week_dates[-1])).strftime("%d-%b").upper()

    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.set_xticks(x_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(y_ticks, crs=ccrs.PlateCarree())
    ax.tick_params(labelbottom=True, labelleft=(i == 0))

    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')

    mesh = ax.pcolormesh(cloud_avg.longitude, cloud_avg.latitude, cloud_avg,
                         cmap="Blues", shading="nearest", vmin=0, vmax=100)

    gdf_peru.boundary.plot(ax=ax, edgecolor='black', linewidth=0.5, zorder=5)
    gdf_suda.plot(ax=ax, color='white', linewidth=0, zorder=10)

    ax.set_title(f"**{semana}**\n{start_date} – {end_date}", fontsize=11)

cbar = fig.colorbar(mesh, ax=axes.ravel().tolist(), orientation="vertical", shrink=0.7, pad=0.03)
cbar.set_label("Cobertura Nubosa Total [%]")

plt.suptitle("NUBOSIDAD SEMANAL PROMEDIO DE ENSEMBLES PARA PERÚ [GEFS]", fontsize=16, y=1.07)
output_file = f"nubes_ensmean_semanal_peru_{pd.to_datetime(latest_init).strftime('%Y%m%d')}.png"
#plt.savefig(output_file, bbox_inches='tight', pad_inches=0.05)
plt.show()
print(f"Imagen guardada como: {output_file}")
